# RAN Domain Model Evaluation

This notebook provides comprehensive evaluation of the fine-tuned RAN domain model.

## Overview
- **Purpose**: Evaluate the trained RAN domain-specific model
- **Prerequisites**: Completed model training from `ran_finetuning.ipynb`
- **Evaluations**: Intent classification, confidence analysis, entity extraction, chatbot integration

## Before Running
Make sure you have:
1. Successfully trained model in `./ran_domain_model/` directory
2. Neo4j connection established
3. Training data available for reference

## 1. Setup and Import Trained Model

In [1]:
# Import required libraries
import sys
import os
import json
import torch
import numpy as np
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import transformers for model loading
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

Python version: 3.10.18 (main, Jul  1 2025, 05:26:40) [GCC 12.2.0]
PyTorch version: 2.7.1+cu126
CUDA available: False


In [2]:
# Import custom modules
sys.path.append('..')
from ran_finetuning import RANDomainModelTrainer
from knowledge_graph_module.kg_builder import RANNeo4jIntegrator
from chatbot import EnhancedRANChatbot

print("✅ Modules imported successfully")

Transformers version: 4.54.1
✅ Modules imported successfully
✅ Modules imported successfully


## 2. Model Configuration and Loading

In [3]:
# Model configuration
MODEL_DIR = './ran_domain_model'  # Path to trained model
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "ranqarag#1"

print(f"📁 Model directory: {MODEL_DIR}")

# Check if model exists
if os.path.exists(MODEL_DIR):
    model_files = os.listdir(MODEL_DIR)
    print(f"✅ Model found with {len(model_files)} files:")
    for file in sorted(model_files):
        file_path = os.path.join(MODEL_DIR, file)
        if os.path.isfile(file_path):
            size_mb = os.path.getsize(file_path) / (1024 * 1024)
            print(f"   {file}: {size_mb:.2f} MB")
else:
    print(f"Model not found at {MODEL_DIR}")
    print("Please run the training notebook first.")

📁 Model directory: ./ran_domain_model
✅ Model found with 10 files:
   config.json: 0.00 MB
   intent_labels.json: 0.00 MB
   model.safetensors: 255.45 MB
   special_tokens_map.json: 0.00 MB
   tokenizer.json: 0.68 MB
   tokenizer_config.json: 0.00 MB
   training_args.bin: 0.01 MB
   training_info.json: 0.00 MB
   vocab.txt: 0.22 MB


In [4]:
# Load the trained model
try:
    # Load classifier pipeline
    classifier = pipeline(
        "text-classification",
        model=MODEL_DIR,
        tokenizer=MODEL_DIR,
        return_all_scores=False
    )
    
    # Load intent labels
    intent_labels_path = f"{MODEL_DIR}/intent_labels.json"
    if os.path.exists(intent_labels_path):
        with open(intent_labels_path, 'r') as f:
            intent_labels = json.load(f)
        print(f"✅ Model loaded with {len(intent_labels)} intent classes")
        
        print("\n🎯 Available Intents:")
        for i, intent in enumerate(intent_labels, 1):
            print(f"   {i:2d}. {intent}")
    else:
        print("⚠️ Intent labels not found, using default mapping")
        intent_labels = list(range(10))  # Fallback
    
    # Load training info if available
    training_info_path = f"{MODEL_DIR}/training_info.json"
    if os.path.exists(training_info_path):
        with open(training_info_path, 'r') as f:
            training_info = json.load(f)
        
        print(f"\n📊 Training Information:")
        print(f"   Base Model: {training_info.get('model_name', 'unknown')}")
        print(f"   Training Samples: {training_info.get('training_samples', 'unknown')}")
        print(f"   Training Date: {training_info.get('training_date', 'unknown')}")
        print(f"   Transformers Version: {training_info.get('transformers_version', 'unknown')}")
    
    model_loaded = True
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    model_loaded = False
    classifier = None
    intent_labels = []

Device set to use cpu


✅ Model loaded with 10 intent classes

🎯 Available Intents:
    1. performance_analysis
    2. power_optimization
    3. spectrum_management
    4. cell_configuration
    5. quality_assessment
    6. traffic_analysis
    7. fault_detection
    8. capacity_planning
    9. interference_analysis
   10. handover_optimization

📊 Training Information:
   Base Model: distilbert-base-uncased
   Training Samples: 1180
   Training Date: 2025-08-08T07:19:55.488820
   Transformers Version: 4.54.1


## 3. Database Connection for Context

In [5]:
# Connect to Neo4j for context
try:
    neo4j_integrator = RANNeo4jIntegrator(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)
    print("✅ Neo4j connection established for evaluation context")
    
    # Quick connection test
    with neo4j_integrator.driver.session() as session:
        result = session.run("MATCH (n) RETURN count(n) as total_nodes")
        total_nodes = result.single()['total_nodes']
        print(f"📊 Knowledge graph nodes: {total_nodes:,}")
        
    neo4j_available = True
    
except Exception as e:
    print(f"⚠️ Neo4j connection failed: {e}")
    print("Evaluation will continue without live KG context")
    neo4j_integrator = None
    neo4j_available = False

✅ Neo4j connection established for evaluation context
📊 Knowledge graph nodes: 5,857
📊 Knowledge graph nodes: 5,857


## 4. Basic Model Testing

In [6]:
# Basic functionality test
if model_loaded:
    print("🧪 Basic Model Functionality Test")
    print("=" * 40)
    
    # Simple test query
    test_query = "Show me power consumption data"
    
    try:
        result = classifier(test_query)
        
        # Extract prediction
        label = result[0]['label']
        confidence = result[0]['score']
        
        # Parse label to intent
        if 'LABEL_' in label:
            label_idx = int(label.split('_')[-1])
            if label_idx < len(intent_labels):
                predicted_intent = intent_labels[label_idx]
            else:
                predicted_intent = f"Unknown_{label_idx}"
        else:
            predicted_intent = label
        
        print(f"Query: '{test_query}'")
        print(f"Predicted Intent: {predicted_intent}")
        print(f"Confidence: {confidence:.3f}")
        
        if confidence > 0.7:
            print("✅ Model is working correctly!")
        else:
            print("⚠️ Low confidence - model may need more training")
            
    except Exception as e:
        print(f"❌ Model test failed: {e}")
        model_loaded = False
else:
    print("⏭️ Skipping basic test - model not loaded")

🧪 Basic Model Functionality Test
Query: 'Show me power consumption data'
Predicted Intent: power_optimization
Confidence: 0.976
✅ Model is working correctly!
Query: 'Show me power consumption data'
Predicted Intent: power_optimization
Confidence: 0.976
✅ Model is working correctly!


## 5. Comprehensive Intent Classification Evaluation

In [8]:
# Comprehensive evaluation with training data-aligned test queries
if model_loaded:
    print("🔍 Comprehensive Intent Classification Evaluation")
    print("=" * 55)
    
    # Test queries aligned with actual training data patterns and RAN entities
    test_queries = [
        # Performance Analysis (intent: performance_analysis, label: 0)
        "Show timing data from NRSynchronization.nRSynchronizationId in NRSynchronization",
        "Get synchronization info for MimoSleepFunction.sleepStartTime",
        "Show timing data from AnrFunction.removeNenbTime in AnrFunction",
        "Display performance metrics from SectorEquipmentFunction",
        "Get performance data for cell timing synchronization",
        
        # Power Optimization (intent: power_optimization, label: 1)
        "Show power data from ConsumedEnergyMeasurement.consumedEnergyMeasurementId",
        "Get power consumption for UeMeasControl.a5TimerUlVolteCovMob",
        "Show power data from HcRule.recommendedAction in HcRule",
        "Display power consumption measurements",
        "Get energy optimization data from cells",
        
        # Spectrum Management (intent: spectrum_management, label: 2)
        "Show frequency data from EUtranFrequency.freqBand",
        "Get spectrum allocation for EUtranFrequency",
        "Show frequency bands from TermPointToGNB",
        "Display spectrum utilization data",
        "Get frequency management information",
        
        # Cell Configuration (intent: cell_configuration, label: 3)
        "Show network topology data from SubscriberGroupProfile.cellTriggerList",
        "Find topology relationships for SharingGroup.gUtranCellRelationRef",
        "Show cell configuration from CellSleepFunction.sleepState",
        "Get cell parameters from AutoCellCapEstFunction.useEstimatedCellCap",
        "Display antenna configuration from Trx.noOfRxAntennas",
        
        # Quality Assessment (intent: quality_assessment, label: 4)
        "Show signal quality from RadioBearerTable.radioBearerTableId",
        "Analyze quality metrics in S1UTermination",
        "Show signal quality from UeMC.userLabel",
        "Get quality measurements from CpriLinkSupervision",
        "Display RSRP and RSRQ values",
        
        # Traffic Analysis (intent: traffic_analysis, label: 5)
        "Show traffic data from ENodeBFunction.eNodeBPlmnId",
        "Get traffic patterns from CapacityUsage.capacityUsageId",
        "Display network load from LoadBalancingFunction",
        "Show data usage statistics",
        "Analyze network traffic distribution",
        
        # Fault Detection (intent: fault_detection, label: 6)
        "Show alarm data from AdmissionControl.admissionControlId",
        "Get fault information from Equipment.equipmentId",
        "Display system faults from HwItem.hwUnitLocation",
        "Show network alarm status",
        "Detect equipment failures",
        
        # Capacity Planning (intent: capacity_planning, label: 7)
        "Show capacity data from AutoCellCapEstFunction.autoCellCapEstFunctionId",
        "Get capacity planning from LoadBalancingFunction.txPwrForOverlaidCellDetect",
        "Display resource utilization forecasts",
        "Show capacity headroom analysis",
        "Plan network expansion capacity",
        
        # Interference Analysis (intent: interference_analysis, label: 8)
        "Show interference data from CellSleepFunction.covCellLatestStatsAdaRatio",
        "Get interference patterns from AnrFunctionNR.gNodebIdLength",
        "Display signal interference levels",
        "Show interference mitigation data",
        "Analyze co-channel interference",
        
        # Handover Optimization (intent: handover_optimization, label: 9)
        "Show handover data from CellSleepFunction.wakeUpLastHoTime",
        "Get handover success rates from X2UTermination.x2UTerminationId",
        "Display mobility performance data",
        "Show handover failure analysis",
        "Optimize handover parameters"
    ]
    
    results = []
    intent_predictions = {}
    
    print(f"Testing {len(test_queries)} training data-aligned queries across all RAN intents...\n")
    
    for i, query in enumerate(test_queries, 1):
        try:
            result = classifier(query)
            
            # Parse result
            label = result[0]['label']
            confidence = result[0]['score']
            
            if 'LABEL_' in label:
                label_idx = int(label.split('_')[-1])
                if label_idx < len(intent_labels):
                    predicted_intent = intent_labels[label_idx]
                else:
                    predicted_intent = f"Unknown_{label_idx}"
            else:
                predicted_intent = label
            
            # Store results
            results.append({
                'Query': query,
                'Predicted_Intent': predicted_intent,
                'Confidence': confidence
            })
            
            # Count predictions by intent
            intent_predictions[predicted_intent] = intent_predictions.get(predicted_intent, 0) + 1
            
            # Display result
            print(f"{i:2d}. Query: '{query[:60]}{'...' if len(query) > 60 else ''}")
            print(f"    Intent: {predicted_intent}")
            print(f"    Confidence: {confidence:.3f}")
            
            # Confidence indicator
            if confidence > 0.8:
                print("    ✅ High confidence")
            elif confidence > 0.6:
                print("    ⚠️ Medium confidence")
            else:
                print("    ❓ Low confidence")
            print()
            
        except Exception as e:
            print(f"{i:2d}. Error processing query: {e}")
            continue
    
    evaluation_completed = True
    
else:
    print("⏭️ Skipping comprehensive evaluation - model not loaded")
    evaluation_completed = False
    results = []
    intent_predictions = {}

🔍 Comprehensive Intent Classification Evaluation
Testing 50 training data-aligned queries across all RAN intents...

 1. Query: 'Show timing data from NRSynchronization.nRSynchronizationId ...
    Intent: performance_analysis
    Confidence: 0.995
    ✅ High confidence

 1. Query: 'Show timing data from NRSynchronization.nRSynchronizationId ...
    Intent: performance_analysis
    Confidence: 0.995
    ✅ High confidence

 2. Query: 'Get synchronization info for MimoSleepFunction.sleepStartTim...
    Intent: performance_analysis
    Confidence: 0.995
    ✅ High confidence

 2. Query: 'Get synchronization info for MimoSleepFunction.sleepStartTim...
    Intent: performance_analysis
    Confidence: 0.995
    ✅ High confidence

 3. Query: 'Show timing data from AnrFunction.removeNenbTime in AnrFunct...
    Intent: performance_analysis
    Confidence: 0.995
    ✅ High confidence

 3. Query: 'Show timing data from AnrFunction.removeNenbTime in AnrFunct...
    Intent: performance_analysis
    

## 6. Statistical Analysis

In [9]:
# Statistical analysis of results
if evaluation_completed and results:
    print("📊 Statistical Analysis of Model Performance")
    print("=" * 45)
    
    # Create DataFrame for analysis
    results_df = pd.DataFrame(results)
    
    # Confidence statistics
    confidences = results_df['Confidence']
    avg_confidence = confidences.mean()
    min_confidence = confidences.min()
    max_confidence = confidences.max()
    std_confidence = confidences.std()
    
    print(f"📈 Confidence Score Analysis:")
    print(f"   Average: {avg_confidence:.3f}")
    print(f"   Range: {min_confidence:.3f} - {max_confidence:.3f}")
    print(f"   Standard Deviation: {std_confidence:.3f}")
    
    # Confidence distribution
    high_conf = len(confidences[confidences > 0.8])
    med_conf = len(confidences[(confidences >= 0.6) & (confidences <= 0.8)])
    low_conf = len(confidences[confidences < 0.6])
    
    print(f"\n📊 Confidence Distribution:")
    print(f"   High confidence (>0.8): {high_conf} queries ({high_conf/len(results)*100:.1f}%)")
    print(f"   Medium confidence (0.6-0.8): {med_conf} queries ({med_conf/len(results)*100:.1f}%)")
    print(f"   Low confidence (<0.6): {low_conf} queries ({low_conf/len(results)*100:.1f}%)")
    
    # Intent distribution
    print(f"\n🎯 Intent Prediction Distribution:")
    sorted_intents = sorted(intent_predictions.items(), key=lambda x: x[1], reverse=True)
    for intent, count in sorted_intents:
        percentage = (count / len(results)) * 100
        print(f"   {intent}: {count} predictions ({percentage:.1f}%)")
    
    # Coverage analysis
    unique_intents = len(set(results_df['Predicted_Intent']))
    total_intents = len(intent_labels)
    coverage = (unique_intents / total_intents) * 100
    
    print(f"\n🎭 Intent Coverage Analysis:")
    print(f"   Unique intents predicted: {unique_intents}/{total_intents} ({coverage:.1f}%)")
    print(f"   Unused intents: {total_intents - unique_intents}")
    
    if unique_intents < total_intents:
        unused_intents = set(intent_labels) - set(results_df['Predicted_Intent'])
        print(f"   Unused intent classes: {', '.join(unused_intents)}")
    
    # Model quality assessment
    print(f"\n🏆 Overall Model Quality Assessment:")
    if avg_confidence > 0.8 and high_conf > len(results) * 0.7:
        print("   ✅ Excellent - High confidence predictions with good coverage")
    elif avg_confidence > 0.7 and high_conf > len(results) * 0.5:
        print("   ✅ Good - Acceptable confidence with reasonable coverage")
    elif avg_confidence > 0.6:
        print("   ⚠️ Fair - May benefit from additional training data")
    else:
        print("   ❌ Poor - Requires more training or data quality improvement")
        
else:
    print("⏭️ Skipping statistical analysis - no evaluation results")

📊 Statistical Analysis of Model Performance
📈 Confidence Score Analysis:
   Average: 0.886
   Range: 0.270 - 0.998
   Standard Deviation: 0.182

📊 Confidence Distribution:
   High confidence (>0.8): 40 queries (80.0%)
   Medium confidence (0.6-0.8): 4 queries (8.0%)
   Low confidence (<0.6): 6 queries (12.0%)

🎯 Intent Prediction Distribution:
   cell_configuration: 17 predictions (34.0%)
   performance_analysis: 10 predictions (20.0%)
   spectrum_management: 8 predictions (16.0%)
   power_optimization: 7 predictions (14.0%)
   quality_assessment: 4 predictions (8.0%)
   traffic_analysis: 3 predictions (6.0%)
   handover_optimization: 1 predictions (2.0%)

🎭 Intent Coverage Analysis:
   Unique intents predicted: 7/10 (70.0%)
   Unused intents: 3
   Unused intent classes: capacity_planning, interference_analysis, fault_detection

🏆 Overall Model Quality Assessment:
   ✅ Excellent - High confidence predictions with good coverage


## 7. RAN-Specific Domain Testing

In [10]:
# Test with RAN-specific terminology and real KG data
if model_loaded and neo4j_available:
    print("🎯 RAN-Specific Domain Testing with Real KG Data")
    print("=" * 52)
    
    # Get some real table/column names from KG for testing
    try:
        with neo4j_integrator.driver.session() as session:
            kg_query = """
                MATCH (t:Table)-[:HAS_COLUMN]->(c:Column)
                RETURN t.name as table_name, c.name as column_name
                LIMIT 10
            """
            result = session.run(kg_query)
            kg_data = [(record['table_name'], record['column_name']) for record in result]
        
        if kg_data:
            print(f"📊 Testing with {len(kg_data)} real KG entities\n")
            
            kg_test_results = []
            
            for i, (table_name, column_name) in enumerate(kg_data, 1):
                # Create realistic RAN queries using real KG data
                test_queries_kg = [
                    f"Show {column_name} from {table_name}",
                    f"Analyze {column_name} patterns",
                    f"What is the {column_name} configuration?",
                    f"Display {table_name} performance data"
                ]
                
                for query in test_queries_kg[:2]:  # Test 2 queries per KG entity
                    try:
                        result = classifier(query)
                        
                        label = result[0]['label']
                        confidence = result[0]['score']
                        
                        if 'LABEL_' in label:
                            label_idx = int(label.split('_')[-1])
                            predicted_intent = intent_labels[label_idx] if label_idx < len(intent_labels) else f"Unknown_{label_idx}"
                        else:
                            predicted_intent = label
                        
                        kg_test_results.append({
                            'Table': table_name,
                            'Column': column_name,
                            'Query': query,
                            'Intent': predicted_intent,
                            'Confidence': confidence
                        })
                        
                        print(f"🏷️ Real KG Query: '{query[:60]}{'...' if len(query) > 60 else ''}")
                        print(f"   Table: {table_name}, Column: {column_name}")
                        print(f"   Predicted: {predicted_intent} ({confidence:.3f})")
                        print()
                        
                    except Exception as e:
                        print(f"   ❌ Error processing KG query: {e}")
                        continue
            
            # Analyze KG-specific results
            if kg_test_results:
                kg_df = pd.DataFrame(kg_test_results)
                kg_avg_conf = kg_df['Confidence'].mean()
                
                print(f"📈 Real KG Data Results:")
                print(f"   Average confidence: {kg_avg_conf:.3f}")
                print(f"   Queries tested: {len(kg_test_results)}")
                print(f"   Unique tables tested: {kg_df['Table'].nunique()}")
                print(f"   Intent variety: {kg_df['Intent'].nunique()} different intents")
                
        else:
            print("⚠️ No KG data available for testing")
            
    except Exception as e:
        print(f"❌ Error accessing KG data: {e}")
        
else:
    print("⏭️ Skipping RAN-specific testing - model or KG not available")

🎯 RAN-Specific Domain Testing with Real KG Data
📊 Testing with 10 real KG entities

🏷️ Real KG Query: 'Show neType from MeContext
   Table: MeContext, Column: neType
   Predicted: cell_configuration (0.996)

🏷️ Real KG Query: 'Analyze neType patterns
   Table: MeContext, Column: neType
   Predicted: cell_configuration (0.998)

🏷️ Real KG Query: 'Show MeContextId from MeContext
   Table: MeContext, Column: MeContextId
   Predicted: cell_configuration (0.998)

🏷️ Real KG Query: 'Analyze MeContextId patterns
   Table: MeContext, Column: MeContextId
   Predicted: cell_configuration (0.998)

🏷️ Real KG Query: 'Show vsDataFormatVersion from MeContext
   Table: MeContext, Column: vsDataFormatVersion
   Predicted: cell_configuration (0.998)

📊 Testing with 10 real KG entities

🏷️ Real KG Query: 'Show neType from MeContext
   Table: MeContext, Column: neType
   Predicted: cell_configuration (0.996)

🏷️ Real KG Query: 'Analyze neType patterns
   Table: MeContext, Column: neType
   Predicted: cel

## 8. Enhanced Chatbot Integration Test

In [11]:
# Test integration with enhanced chatbot
if model_loaded and neo4j_available:
    print("🤖 Enhanced Chatbot Integration Evaluation")
    print("=" * 43)
    
    try:
        # Initialize enhanced chatbot with fine-tuned model
        enhanced_chatbot = EnhancedRANChatbot(
            neo4j_integrator, 
            use_domain_model=True
        )
        
        print("✅ Enhanced chatbot initialized with fine-tuned model\n")
        
        # Training data-aligned chatbot test queries
        chatbot_test_queries = [
            # Real patterns from training data
            "Show power data from ConsumedEnergyMeasurement.consumedEnergyMeasurementId",
            "Get network topology from SubscriberGroupProfile.cellTriggerList",
            "Find signal quality from RadioBearerTable.radioBearerTableId",
            "Show frequency data from EUtranFrequency.freqBand", 
            "Display timing data from NRSynchronization.nRSynchronizationId",
            "Get handover data from CellSleepFunction.wakeUpLastHoTime",
            
            # Variations that should map to training patterns
            "Show cell sleep configuration parameters",
            "Get power consumption measurements from base stations",
            "Find interference analysis data",
            "Display spectrum allocation information",
            "Show performance metrics for eNodeB functions",
            "Get quality assessment for radio bearers"
        ]
        
        chatbot_results = []
        
        for i, query in enumerate(chatbot_test_queries, 1):
            print(f"🔍 Test {i}: {query}")
            print("-" * 60)
            
            try:
                result = enhanced_chatbot.enhanced_process_query(query)
                
                # Extract key information
                query_type = result.get('type', 'unknown')
                detected_intent = result.get('intent', 'unknown')
                entities = result.get('entities', {})
                response = result.get('response', 'No response')
                
                print(f"Query Type: {query_type}")
                print(f"Detected Intent: {detected_intent}")
                
                # Show entities summary with RAN-specific focus
                if entities:
                    entity_summary = []
                    ran_entities = ['table_name', 'column_name', 'network_element', 'measurement_type']
                    
                    for entity_type, values in entities.items():
                        if values and entity_type in ran_entities:
                            if isinstance(values, list):
                                entity_summary.append(f"{entity_type}: {', '.join(str(v) for v in values[:2])}")
                            else:
                                entity_summary.append(f"{entity_type}: {values}")
                    
                    if entity_summary:
                        print(f"RAN Entities: {'; '.join(entity_summary[:3])}")
                    else:
                        print(f"General Entities: {len(entities)} detected")
                
                # Show response preview
                response_preview = response[:120] + "..." if len(response) > 120 else response
                print(f"Response Preview: {response_preview}")
                
                # Evaluate alignment with training data patterns
                alignment_score = 0
                if any(keyword in query.lower() for keyword in ['show', 'get', 'find', 'display']):
                    alignment_score += 1
                if any(pattern in query for pattern in ['.', 'from', 'in']):
                    alignment_score += 1
                if detected_intent in intent_labels:
                    alignment_score += 1
                
                alignment_status = "High" if alignment_score >= 2 else "Medium" if alignment_score == 1 else "Low"
                print(f"Training Alignment: {alignment_status} ({alignment_score}/3)")
                
                # Store result for analysis
                chatbot_results.append({
                    'Query': query,
                    'Type': query_type,
                    'Intent': detected_intent,
                    'Has_Entities': len(entities) > 0,
                    'Has_Response': len(response) > 0,
                    'Response_Length': len(response),
                    'Training_Alignment': alignment_score
                })
                
                print("✅ Success")
                
            except Exception as e:
                print(f"❌ Error processing query: {e}")
                chatbot_results.append({
                    'Query': query,
                    'Type': 'error',
                    'Intent': 'error',
                    'Has_Entities': False,
                    'Has_Response': False,
                    'Response_Length': 0,
                    'Training_Alignment': 0
                })
            
            print("\n")
        
        # Analyze chatbot integration results with training data focus
        if chatbot_results:
            chatbot_df = pd.DataFrame(chatbot_results)
            
            success_rate = len(chatbot_df[chatbot_df['Type'] != 'error']) / len(chatbot_df) * 100
            entity_rate = len(chatbot_df[chatbot_df['Has_Entities']]) / len(chatbot_df) * 100
            response_rate = len(chatbot_df[chatbot_df['Has_Response']]) / len(chatbot_df) * 100
            avg_alignment = chatbot_df['Training_Alignment'].mean()
            
            print(f"📊 Enhanced Chatbot Integration Summary:")
            print(f"   Success Rate: {success_rate:.1f}%")
            print(f"   Entity Extraction: {entity_rate:.1f}%")
            print(f"   Response Generation: {response_rate:.1f}%")
            print(f"   Training Data Alignment: {avg_alignment:.1f}/3.0")
            print(f"   Average Response Length: {chatbot_df['Response_Length'].mean():.0f} characters")
            
            # Intent distribution analysis
            intent_counts = chatbot_df['Intent'].value_counts()
            print(f"\n🎯 Intent Distribution:")
            for intent, count in intent_counts.head(5).items():
                percentage = (count / len(chatbot_df)) * 100
                print(f"   {intent}: {count} queries ({percentage:.1f}%)")
            
            # Training alignment assessment
            high_alignment = len(chatbot_df[chatbot_df['Training_Alignment'] >= 2])
            if high_alignment >= len(chatbot_df) * 0.7:
                print(f"\n✅ Excellent training data alignment ({high_alignment}/{len(chatbot_df)} queries)")
            elif high_alignment >= len(chatbot_df) * 0.5:
                print(f"\n⚠️ Good training data alignment ({high_alignment}/{len(chatbot_df)} queries)")
            else:
                print(f"\n❓ Moderate training data alignment ({high_alignment}/{len(chatbot_df)} queries)")
        
        print("\n✅ Enhanced chatbot integration evaluation completed")
        
    except ImportError as e:
        print(f"❌ Cannot import enhanced chatbot: {e}")
        print("Make sure the chatbot module is available")
    except Exception as e:
        print(f"❌ Error testing enhanced chatbot: {e}")
        
else:
    print("⏭️ Skipping chatbot integration test")
    print("Requirements: trained model + Neo4j connection")

🤖 Enhanced Chatbot Integration Evaluation
Domain-specific model loaded successfully
✅ Enhanced chatbot initialized with fine-tuned model

🔍 Test 1: Show power data from ConsumedEnergyMeasurement.consumedEnergyMeasurementId
------------------------------------------------------------
Query Type: domain_inquiry
Detected Intent: unknown
Response Preview: No response
Training Alignment: High (2/3)
✅ Success


🔍 Test 2: Get network topology from SubscriberGroupProfile.cellTriggerList
------------------------------------------------------------
Query Type: domain_inquiry
Detected Intent: unknown
Response Preview: No response
Training Alignment: High (2/3)
✅ Success


🔍 Test 2: Get network topology from SubscriberGroupProfile.cellTriggerList
------------------------------------------------------------
Query Type: domain_inquiry
Detected Intent: unknown
Response Preview: No response
Training Alignment: High (2/3)
✅ Success


🔍 Test 3: Find signal quality from RadioBearerTable.radioBearerTableI

## 9. Interactive Testing Playground

In [12]:
# Interactive testing - you can modify and run this cell multiple times
if model_loaded:
    print("🧪 Interactive Model Testing Playground")
    print("=" * 42)
    print("Add your own test queries below and run this cell to test them:")
    print()
    
    # Training data-aligned custom test queries (modify these to test your own patterns)
    custom_queries = [
        # Examples based on actual training data patterns:
        "Show power data from ConsumedEnergyMeasurement in base stations",
        "Get network topology from CellSleepFunction.sleepState",
        "Find interference sources in AnrFunctionNR.gNodebIdLength",
        "Display handover performance from mobility functions",
        "Show quality metrics from RadioBearerTable configuration",
        "Get spectrum allocation from EUtranFrequency.freqBand data",
        
        # Add your own custom queries here (training data style):
        # "Show [data_type] from [Table].[column] in [context]",
        # "Get [measurement] for [specific_entity]",
        # "Display [performance_metric] from [network_element]",
        # "Find [relationship] in [topology_element]",
        # "Analyze [quality_aspect] metrics in [component]",
    ]
    
    if not custom_queries or all(not q.strip() for q in custom_queries):
        print("💡 Add some queries to the custom_queries list above and re-run this cell")
        print("💡 Use patterns like: 'Show [data] from [Table].[column]' or 'Get [info] for [entity]'")
    else:
        print("🎯 Training Data-Style Query Testing:")
        print("   Testing queries that match our RAN training data patterns...")
        print()
        
        for i, query in enumerate(custom_queries, 1):
            if query.strip():  # Skip empty queries
                try:
                    result = classifier(query)
                    
                    label = result[0]['label']
                    confidence = result[0]['score']
                    
                    if 'LABEL_' in label:
                        label_idx = int(label.split('_')[-1])
                        predicted_intent = intent_labels[label_idx] if label_idx < len(intent_labels) else f"Unknown_{label_idx}"
                    else:
                        predicted_intent = label
                    
                    print(f"{i}. Query: '{query}'")
                    print(f"   🎯 Intent: {predicted_intent}")
                    print(f"   📊 Confidence: {confidence:.3f}")
                    
                    # Quality indicator with training data context
                    if confidence > 0.8:
                        print("   ✅ High confidence - Well aligned with training patterns")
                    elif confidence > 0.6:
                        print("   ⚠️ Medium confidence - Partially matches training patterns")
                    else:
                        print("   ❓ Low confidence - May need more similar training examples")
                    
                    # Show expected intent category based on keywords
                    query_lower = query.lower()
                    if any(keyword in query_lower for keyword in ['power', 'energy', 'consumption']):
                        print("   📝 Expected: power_optimization (based on keywords)")
                    elif any(keyword in query_lower for keyword in ['topology', 'cell', 'configuration']):
                        print("   📝 Expected: cell_configuration (based on keywords)")
                    elif any(keyword in query_lower for keyword in ['quality', 'signal', 'rsrp', 'rsrq']):
                        print("   📝 Expected: quality_assessment (based on keywords)")
                    elif any(keyword in query_lower for keyword in ['handover', 'mobility']):
                        print("   📝 Expected: handover_optimization (based on keywords)")
                    elif any(keyword in query_lower for keyword in ['spectrum', 'frequency', 'band']):
                        print("   📝 Expected: spectrum_management (based on keywords)")
                    elif any(keyword in query_lower for keyword in ['interference']):
                        print("   📝 Expected: interference_analysis (based on keywords)")
                    
                    print()
                    
                except Exception as e:
                    print(f"{i}. Error testing query '{query}': {e}")
                    print()
        
        print("💡 Tips for better alignment with training data:")
        print("   • Use specific table.column patterns like 'EUtranFrequency.freqBand'")
        print("   • Include context like 'Show X data from Y in Z'")
        print("   • Reference actual RAN entities from the knowledge graph")
        print("   • Use domain-specific terminology (eNodeB, gNodeB, RSRP, etc.)")
        
else:
    print("⏭️ Model not ready for interactive testing")
    print("Please ensure the model is loaded successfully first.")

🧪 Interactive Model Testing Playground
Add your own test queries below and run this cell to test them:

🎯 Training Data-Style Query Testing:
   Testing queries that match our RAN training data patterns...

1. Query: 'Show power data from ConsumedEnergyMeasurement in base stations'
   🎯 Intent: power_optimization
   📊 Confidence: 0.986
   ✅ High confidence - Well aligned with training patterns
   📝 Expected: power_optimization (based on keywords)

2. Query: 'Get network topology from CellSleepFunction.sleepState'
   🎯 Intent: cell_configuration
   📊 Confidence: 0.998
   ✅ High confidence - Well aligned with training patterns
   📝 Expected: cell_configuration (based on keywords)

3. Query: 'Find interference sources in AnrFunctionNR.gNodebIdLength'
   🎯 Intent: spectrum_management
   📊 Confidence: 0.824
   ✅ High confidence - Well aligned with training patterns
   📝 Expected: interference_analysis (based on keywords)

4. Query: 'Display handover performance from mobility functions'
   🎯 

## 10. Evaluation Summary and Recommendations

In [13]:
# Final evaluation summary
print("📋 RAN Domain Model Evaluation Summary")
print("=" * 42)
print(f"📅 Evaluation completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

# Summarize evaluation status
evaluation_status = [
    ("Model Loading", "✅ Success" if model_loaded else "❌ Failed"),
    ("Neo4j Connection", "✅ Connected" if neo4j_available else "⚠️ Unavailable"),
    ("Basic Functionality", "✅ Working" if model_loaded else "❌ Failed"),
    ("Comprehensive Testing", "✅ Completed" if evaluation_completed else "❌ Skipped"),
]

print("🔍 Evaluation Components:")
for component, status in evaluation_status:
    print(f"   {component}: {status}")
print()

if evaluation_completed and results:
    # Performance summary
    avg_conf = pd.DataFrame(results)['Confidence'].mean()
    high_conf_pct = len([r for r in results if r['Confidence'] > 0.8]) / len(results) * 100
    
    print("📊 Performance Metrics:")
    print(f"   Average Confidence: {avg_conf:.3f}")
    print(f"   High Confidence Rate: {high_conf_pct:.1f}%")
    print(f"   Total Queries Tested: {len(results)}")
    print(f"   Intent Coverage: {len(set(r['Predicted_Intent'] for r in results))}/{len(intent_labels)} intents")
    print()

# Recommendations
print("💡 Recommendations:")

if model_loaded:
    if evaluation_completed and avg_conf > 0.8:
        print("   🎉 Model performs excellently! Ready for production use.")
        print("   📈 Consider monitoring real-world performance and collecting user feedback.")
    elif evaluation_completed and avg_conf > 0.7:
        print("   ✅ Model performs well. Consider additional training for edge cases.")
        print("   🔄 Monitor low-confidence predictions for potential improvement areas.")
    elif evaluation_completed:
        print("   ⚠️ Model shows moderate performance. Consider:")
        print("     • Adding more diverse training data")
        print("     • Increasing training epochs")
        print("     • Fine-tuning hyperparameters")
    
    if neo4j_available:
        print("   🔗 Excellent - Full integration with knowledge graph available.")
    else:
        print("   ⚠️ Consider establishing Neo4j connection for full functionality.")
        
    print("   📚 Next steps:")
    print("     • Deploy model in production chatbot")
    print("     • Set up monitoring and logging")
    print("     • Collect user feedback for continuous improvement")
    print("     • Schedule periodic retraining with new data")
    
else:
    print("   ❌ Model evaluation incomplete. Please:")
    print("     • Check model training completed successfully")
    print("     • Verify model files exist in the expected directory")
    print("     • Review any error messages above")
    print("     • Re-run training if necessary")

print("\n🎯 For production deployment:")
print("   • Test with real user queries")
print("   • Implement confidence thresholds")
print("   • Set up fallback mechanisms for low-confidence predictions")
print("   • Monitor performance metrics continuously")
print("\n✅ Evaluation completed!")

📋 RAN Domain Model Evaluation Summary
📅 Evaluation completed at: 2025-08-08 13:07:05

🔍 Evaluation Components:
   Model Loading: ✅ Success
   Neo4j Connection: ✅ Connected
   Basic Functionality: ✅ Working
   Comprehensive Testing: ✅ Completed

📊 Performance Metrics:
   Average Confidence: 0.886
   High Confidence Rate: 80.0%
   Total Queries Tested: 50
   Intent Coverage: 7/10 intents

💡 Recommendations:
   🎉 Model performs excellently! Ready for production use.
   📈 Consider monitoring real-world performance and collecting user feedback.
   🔗 Excellent - Full integration with knowledge graph available.
   📚 Next steps:
     • Deploy model in production chatbot
     • Set up monitoring and logging
     • Collect user feedback for continuous improvement
     • Schedule periodic retraining with new data

🎯 For production deployment:
   • Test with real user queries
   • Implement confidence thresholds
   • Set up fallback mechanisms for low-confidence predictions
   • Monitor performanc